In [12]:
import datetime
import time
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

def train_model(X_train, y_train):
    # Crear el pipeline de ingeniería de características y modelo
    pipeline = Pipeline([
        # Agrega aquí tus etapas de preprocesamiento y transformación de características
        ('scaler', StandardScaler()),
        ('random_forest', RandomForestClassifier())
    ])

    # Iniciar contador de tiempo
    start_time = time.time()

    # Entrenar el modelo
    pipeline.fit(X_train, y_train)

    # Calcular el tiempo de entrenamiento en segundos
    training_time = time.time() - start_time

    # Obtener las predicciones en los datos de entrenamiento
    y_pred = pipeline.predict(X_train)

    # Calcular las métricas de entrenamiento
    accuracy = accuracy_score(y_train, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    roc_auc = roc_auc_score(y_train, y_pred)

    # Generar el archivo de salida con los resultados
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_file = f"train_results_{timestamp}.txt"
    with open(output_file, "w") as file:
        file.write("Fecha y hora de ejecución: {}\n".format(timestamp))
        file.write("Tiempo de entrenamiento: {:.2f} segundos\n".format(training_time))
        file.write("Accuracy: {:.4f}\n".format(accuracy))
        file.write("Specificity: {:.4f}\n".format(specificity))
        file.write("Sensitivity: {:.4f}\n".format(sensitivity))
        file.write("ROC-AUC: {:.4f}\n".format(roc_auc))

    print("Entrenamiento completado. Resultados guardados en {}".format(output_file))


def predict(X_test):
    # Cargar el modelo entrenado
    pipeline = Pipeline([
        # Agrega aquí tus etapas de preprocesamiento y transformación de características
        ('scaler', StandardScaler()),
        ('random_forest', RandomForestClassifier())
    ])
    pipeline.fit(X_train, y_train)

    # Realizar las predicciones en los datos de prueba
    y_pred = pipeline.predict(X_test)

    # Generar el archivo CSV con las predicciones
    predictions_df = pd.DataFrame(y_pred, columns=["Prediction"])
    predictions_df.to_csv("predictions.csv", index=False)

    print("Predicciones generadas. Resultados guardados en predictions.csv")


# Cargar los datos de entrenamiento y prueba
dataTrain = pd.read_excel("DB4.xlsx")
cols_con_na = [col for col in dataTrain.columns if (dataTrain[col].isnull().mean() < 0.05)]
nuevo_df = dataTrain.loc[:, cols_con_na]
data_numericas = nuevo_df.select_dtypes(include=['float', 'int'])
df = nuevo_df

def getDateColTypes(df):
    categoricas = []
    continuas = []
    discretas = []

    for colName in df.columns:
        if df[colName].dtype == "O":
            categoricas.append(colName)
        else:
            if((df[colName].dtype == "int64" or df[colName].dtypes == 'float64')):
                if (len(df[colName]) <=30):
                    discretas.append(colName)
                else:
                    continuas.append(colName)

    return discretas, continuas, categoricas

discretas, continuas, categoricas = getDateColTypes(df)
categoricas_df = dataTrain.loc[:, categoricas]
continuas_df = dataTrain.loc[:, continuas]

def imputar_media(datos, columna):
    media = datos[columna].mean()
    datos_imputados = datos.copy()
    datos_imputados[columna].fillna(media, inplace=True)
    return datos_imputados

for columna in continuas:
    datos_imputados_media = imputar_media(df, columna)

def imputar_categorico(datos):
    datos_imputados = categoricas_df.copy()
    for columna in categoricas_df.columns:
        moda = categoricas_df[columna].mode().iloc[0]
        datos_imputados[columna].fillna(moda, inplace=True)
    return datos_imputados

df_imputado_cat = imputar_categorico(categoricas_df)

label_encoder = LabelEncoder()

df_imputado_cat['mes_codificado'] = label_encoder.fit_transform(df_imputado_cat['mes_ocu']) + 1
df_imputado_cat['vehiculo_codificado'] = label_encoder.fit_transform(df_imputado_cat['tipo_vehi']) + 1
df_imputado_cat['ocu_codificado'] = label_encoder.fit_transform(df_imputado_cat['lesio_fall']) + 1
df_imputado_cat['causa_codificado'] = label_encoder.fit_transform(df_imputado_cat['Causa_acc']) + 1
df_imputado_cat['depto_codificado'] = label_encoder.fit_transform(df_imputado_cat['depto_ocu']) + 1
df_imputado_cat['dia_codificado'] = label_encoder.fit_transform(df_imputado_cat['dia_sem_ocu']) + 1
df_imputado_cat['mes_codificado'] = label_encoder.fit_transform(df_imputado_cat['mes_ocu']) + 1

df_copia = df_imputado_cat.copy()

columnas_eliminar = ['mes_ocu', 'dia_sem_ocu', 'depto_ocu', 'tipo_vehi', 'Causa_acc']

df_copia = df_copia.drop(columnas_eliminar, axis=1)

dataset = df

def getColumnTypes(dataset):
    continuas = []
    discretas = []
    categoricas = []

    for col in dataset.columns:
        if((dataset[col].dtype == 'int64') or (dataset[col].dtype == 'float64')):
            if(len(dataset[col].unique())>30):
                continuas.append(col)
            else:
                discretas.append(col)
        else:
            categoricas.append(col)
    return continuas, discretas, categoricas

df_copia2 = df_copia.drop('lesio_fall', axis=1)

df3 = pd.concat([df_copia2, data_numericas], axis=1)
df3.shape

dfc2 = df3.copy()

df_cleaned = dfc2.dropna()

X = df_cleaned.drop(['dia_codificado', 'year', 'vehiculo_codificado', 'mes_codificado'], axis=1)
y = df_cleaned['dia_codificado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=125)

# Llamar a la función de entrenamiento
train_model(X_train, y_train)

# Llamar a la función de predicción
predict(X_test)


ValueError: too many values to unpack (expected 4)